# 所需库

In [234]:
from PyQt5 import QtWidgets,QtCore, QtGui
from PyQt5.QtCore import QSize, Qt
from PyQt5.QtWidgets import *
from PyQt5.QtGui import QPicture,QPainter
from PyQt5.QtCore import QPointF,QRectF
import sys
import pyqtgraph as pg
import time
from datetime import datetime
import pandas as pd
import numpy as np

# 获取数据

In [235]:
def get_stock_history_from_yahoo(ticker):
    urlstr = "https://finance.yahoo.com/quote/{0}/history?p={0}".format(ticker) 
    hdata = pd.read_html(urlstr)[0][:-1] 
    hdata = hdata.set_index('Date') 
    hdata.index = pd.to_datetime(hdata.index)
    hdata = hdata.astype('d')
    hdata.columns = ['Open','High','Low','Close', 'AdjClose','Volume'] 
    hdata_reindexed=hdata.reset_index()
    return hdata_reindexed

In [236]:
nflx=get_stock_history_from_yahoo('NFLX')

In [237]:
valid_set=nflx.sort_values(by='Date').reset_index(drop=True)

In [238]:
valid_set

,Date,Open,High,Low,Close,AdjClose,Volume
0,2020-02-26,366.31,382.00,365.00,379.24,379.24,8934100.0
1,2020-02-27,371.46,391.56,370.60,371.71,371.71,10949100.0
2,2020-02-28,364.21,376.77,356.80,369.03,369.03,11178600.0
3,2020-03-02,373.11,381.36,364.50,381.05,381.05,6997900.0
4,2020-03-03,381.03,393.52,367.40,368.77,368.77,8364600.0
...,...,...,...,...,...,...,...
95,2020-07-13,567.98,575.37,520.96,525.50,525.50,18399000.0
96,2020-07-14,517.08,525.50,490.49,524.88,524.88,15083300.0
97,2020-07-15,516.30,529.00,510.18,523.26,523.26,10066700.0
98,2020-07-16,526.48,535.54,504.36,527.39,527.39,24499000.0


In [299]:
x_axis=valid_set.index
real_axis=list(valid_set.Date)
real_axis[0]
datetime.fromtimestamp(datetime.timestamp(real_axis[0]))

datetime.datetime(2020, 2, 26, 0, 0)

# 绘制图形 item

In [240]:
class DrawRecItem(pg.GraphicsObject):
    def __init__(self,data):
        super().__init__()
        self.data=data
        self.draw_rect()
        
    def draw_rect(self):
        
        self.picture=QPicture()
        p1=QPainter(self.picture)
        
        # 设置画pen 颜色，用来画线
        p1.setPen(pg.mkPen('w'))
        for i in range(len(self.data)):
            #画一条最大值最小值之间的线
            p1.drawLine(QPointF(self.data.index[i],self.data.Low[i]),QPointF(self.data.index[i],self.data.High[i]))
            # 设置画刷颜色
            if self.data.Open[i]>self.data.Close[i]:
                p1.setBrush(pg.mkBrush('g'))
            else:
                p1.setBrush(pg.mkBrush('r'))
            p1.drawRect(QRectF(self.data.index[i]-0.3,self.data.Open[i],0.6,self.data.Close[i]-self.data.Open[i]))
         
        
        
    def paint(self,p,*args):
        p.drawPicture(0,0,self.picture)
    def boundingRect(self):
        return QRectF(self.picture.boundingRect())


In [241]:
temp=[1,2,4,7,2,3]
temp[-1]

3

# 时间轴

In [315]:
class MyAxisItem(pg.AxisItem):
    def __init__(self,ticks,*args,**kwargs):
        pg.AxisItem.__init__(self,*args,**kwargs)
        self.x_values=[x[0] for x in ticks]
        self.x_strings=[x[1] for x in ticks]
    
    def tickStrings(self, values, scale, spacing):
        strings=[]
        for v in values:
            vs=v*scale
            if vs in self.x_values:
                vstr=self.x_strings[np.abs(self.x_values-vs).argmin()]
                temp_datetime=datetime.fromtimestamp(datetime.timestamp(vstr))
                temp_string=temp_datetime.strftime("%Y-%m-%d %H:%M:%S")
                vstr=temp_string.split()[0]
            else:
                vstr=''
            
            strings.append(vstr)
        return strings

# 主界面

In [316]:
class Control_sys_Tab(QTabWidget):
    def __init__(self,parent=None):
#         self.RequestRowKey ={}
#         self.OrderRowKey  ={}
#         self.BatchRowKey  ={}   #To manage batch row index
#         self.ErrorRowKey  ={}   #To manage error row index
#         self.BuyBatchValue = {}
#         self.SellBatchValue= {}
#         self.BatchManagers  ={} # To manage value for each batch
#         self.g_CurrRequestRow = 0
#         self.g_CurrOrderRow  = 0
#         self.g_CurrBatchRow  = 0
#         self.g_CurrErrorRow  = 0
        super().__init__(parent)
        
        self.setObjectName("Control_system")
        self.resize(1800,985)
        self.setWindowTitle("实时监控系统")
        self.Data=valid_set
        
        #创建3个选项卡小控件窗口
        self.tab1=QWidget()
        self.tab2=QWidget()
        self.tab3=QWidget()
        
        
        #将三个选项卡添加到顶层窗口中
        self.addTab(self.tab1, "交易界面")
        self.addTab(self.tab2, "PNL展示界面")
        self.addTab(self.tab3, "PNL定时发送")

        #每个选项卡自定义的内容
        self.tab1UI()
        self.tab2UI()
        self.tab3UI()
        
        
    def tab1UI(self):

        return 

    def tab2UI(self):
        layout=QVBoxLayout()
        item=DrawRecItem(self.Data)

#         # 设置坐标轴
#         x_axis=valid_set.index
#         real_axis=valid_set.Date
#         axis_min=datetime.timestamp(real_axis[0])
#         axis_max=datetime.timestamp(real_axis[len(real_axis)-1])
#         ticks=[(i,datetime.timestamp(j)) for i,j in zip(x_axis,real_axis)]
#         strAxis=pg.DateAxisItem()
#         strAxis.setTicks([ticks])
        ticks=[(i,j) for i ,j in zip(self.Data.index,self.Data.Date)]
        strAxis=MyAxisItem(ticks,orientation="bottom")
        
        self.plt=pg.PlotWidget(axisItems = {'bottom': strAxis})
#         #self.plt.setXRange(axis_min,axis_max)
        
        #将iTem加入到plotwidget控件中
        self.plt.addItem(item)

        #将控件添加到pyqt中
        layout.addWidget(self.plt)
        #将layout 布局添加到 tab2中
        self.tab2.setLayout(layout)
    def tab3UI(self):

        return 




# Main

In [ ]:
if __name__ == "__main__":
    app = QtWidgets.QApplication.instance()
    if app is None:
        app = QtWidgets.QApplication(sys.argv)
    w = Control_sys_Tab()
    w.show()
    sys.exit(app.exec_())